# Introduction to Graph Neural Networks (GNNs)

In traditional machine learning approaches, <b>structured data</b> is often encountered in the form of <b>sequences</b> or <b>images</b>, represented by one-dimensional and two-dimensional arrays of variables, respectively. However, there exists a broader class of structured data that is best described by a <b>graph</b>. 


A graph consists of a set of objects, called <b>nodes</b>, connected by <b>edges</b>, where both nodes and edges can carry associated data. For instance, in the context of molecules, nodes represent atoms (such as carbon, nitrogen, hydrogen, etc.), and edges correspond to bonds (single, double, etc.). Similarly, in a rail network, nodes represent cities, and edges represent railway lines, often associated with variables like average journey time between cities.

![title](images/graphs.png)

### How can we apply ML (and DL) to graph-structured data?

<p style="text-align:center;"><img src="images/image-graph.png" alt="drawing" width="400" style="text-align:center;"/> </p><br/>
An <b>image</b> is a special instance of graph-structured data in which the <b>nodes</b> are the <b>pixels</b> and the <b>edges</b> describe <b>which pixels are adjacent</b>.

<p style="text-align:center;"><img src="images/vs.png" alt="drawing" width="800" style="text-align:center;"/> </p><br/>

Convolutional neural networks (CNNs) take this structure into account, incorporating prior knowledge of the relative positions of the pixels, and can be used as a source of inspiration to construct
more general approaches to deep learning for graphical data, known as <b>Graph Neural Networks (GNNs)</b> <br/> 

<i>(Zhou et al., 2018; Wu et al., 2019; Hamilton, 2020; Velickovic', 2023)</i>

### Which tasks can be solved by using GNNs?

### Graph-level tasks

In a graph-level task, our goal is to predict the property/label of an <b>entire graph</b>. For example, for a molecule represented as a graph, we might wish to predict if it is soluble in water.


<p style="text-align:center;"><img src="images/gc.png" alt="drawing" width="800" style="text-align:center;"/> </p>

This is analogous to image classification problems with MNIST and CIFAR, where we want to associate a label to an entire image. With text, a similar problem is sentiment analysis where we want to identify the mood or emotion of an entire sentence at once.

### Node-level tasks

Node-level tasks involve predicting a label <b>for each node</b> within a graph. A classic example is Zach's karate club, where individuals align with one of two clubs after a rift between Mr. Hi and John A.

Nodes represent practitioners, edges signify interactions, and the goal is to classify whether a member stands with Mr. Hi or John A.

<p style="text-align:center;"><img src="images/nl.png" alt="drawing" width="800" style="text-align:center;"/> </p>

Following the image analogy, node-level prediction problems are analogous to image segmentation, where we are trying to label the role of each pixel in an image.

This was an example of <i>inductive</i> learning. However, some graph prediction examples are <i>transductive</i> in which we are given the structure of the entire graph along with labels for some of the nodes and the goal is to predict the labels of the remaining nodes. 

An example would be a large social network in which our goal is to classify each node as either a real person or an automated bot. Here a small number of nodes might be manually labelled, but it would be prohibitive to investigate every node individually in a large and ever-changing social network. During training, we therefore have access to the whole graph along with labels for a subset of the nodes, and we wish to predict the labels for the remaining nodes. 

This can be viewed as a form of <b>semi-supervised learning</b>.

### Edge-level tasks

Node-level tasks involve predicting a label <b>for each edge</b> within a graph. One example is in image scene understanding, where models can predict relationships between objects.

By initially assuming a fully connected graph and pruning edges based on predictions, connections between entities can be discovered effectively.

<p style="text-align:center;"><img src="images/el.png" alt="drawing" width="800" style="text-align:center;"/> </p>


### Notation

Generic graph: $$G(V, E)$$
Neighborhood of vertex $V$: $$\mathcal{N}(v_i): (v_j \, | \, v_j \in (v_i, v_j) \subseteq V \times V)$$

For each node $v$ we can represent node variables as a $D$-dimensional column vector $h(v)$, which can be grouped in a matrix $$H\in \mathbb{R}^{|V|\times D}$$ Also edges features can be taken into account (for simplicty let's constraint us to only node features)

### Adjacency matrix

A convenient way to represent edges in a graph is to use an <b>adjacency matrix</b> denoted by $A\in \{0,1\}^{|V|\times|V|}$<br/>

<p style="text-align:center;"><img src="images/adj_vera.png" alt="drawing" width="800" style="text-align:center;"/> </p>

Note that for undirected graphs $A$ will be <b>symmetric</b>.

A first attempt to learn a graph structure could be using directly $A$ as the input for a neural network, for example by stacking $A$ column vectors in a single flattened vector. However this graph representation strictly depends on <b>node ordering</b>!

<p style="text-align:center;"><img src="images/adj.png" alt="drawing" width="800" style="text-align:center;"/> </p>

Phisically trying to learn permutation invariance consists of going through a heavy data augmentation pipeline. Instead we should treat this invariance property as an <b>inductive bias</b> when constructing a network architecture.

### Invariance

In mathematical terms invariance can be expressed as:
$$y(\tilde{H},\tilde{A})=y(H,A)$$
<p>where $\tilde{H}=PH$, and $\tilde{A}=PAP^T$, with $P$ being an arbitrary <b>permutation matrix</b>, that is a matrix composed of $|V|$ <i>standard unit vectors</i>.</p>
$y(\cdot, \cdot)$ is the network output.
<p style="text-align:center;"><img src="images/perm_1.png" alt="drawing" width="800" style="text-align:center;"/> </p>

### Equivariance

In mathematical terms equivariance can be expressed as:
$$y(\tilde{H},\tilde{A})=Py(H,A)$$
<p>where $\tilde{H}=PH$, and $\tilde{A}=PAP^T$, with $P$ being an arbitrary <b>permutation matrix</b>, that is a matrix composed of $|V|$ <i>standard unit vectors</i>.</p>
$y(\cdot, \cdot)$ is the network output.

## Neural Message-Passing

### Requirements for implementation of a MP-Layer

<ul>
    <li><b>Invariance</b> to node permutation</li>
    <li>Handling of <b>variable-length</b> inputs
    <li>Scalable models (graphs can have many nodes)</li>
</ul>

### Convolutional filters

To meet all these requirements, we can seek inspiration from CNNs (like we've seen before).
<p>An image can be seen as a specific instance of graph-structured data, in which <i>nodes</i> are the pixels and edges represent pairs of pixels that are adjacent (also in diagonal direction).</p>
<p>In a CNN, we make successive transformations of trhe image domain such that a pixel at a particular layer computes a function of states of pixels in the previous layer through a local function called <b>filter</b>.

<p style="text-align:center;"><img src="images/conv.png" alt="drawing" width="600" style="text-align:center;"/> </p>
The computation performed by a single filter at a single pixel in layer $l+1$ can be expressed as:
$$h_i^{(l+1)}=f\left(\sum_j w_j h_j^{(l)}\right)$$
where $f$ is a differentiable nonlinear activation function such as a ReLU.
<p>The same function is applied across all patches in the image, so that weights $w_j$ are shared across the patches.</p>

As it stands $h_i^{(l+1)}=f\left(\sum_j w_j h_j^{(l)}\right)$ is not equivariant under reordering of nodes in layer $l$.
<p>We can achieve equivariance with some simple modifications. We first consider also the filter as a graph and separate out the contribution from node $i$. The other eight nodes are its $\mathcal{N}(i)$. We then assume that a <b>single</b> weight parameter $w_{neigh}$ is shared across neighbors such that</p>
$$h_i^{(l+1)}=f\left(w_{neigh}\sum_{j_in\mathcal{N}(i)} h_j^{(l)}+w_{self}h_i^{(l)}\right)$$
where node $i$ has its own parameter $w_{self}$.
<p>$h_j^{(l)}$ can be considered as <i>messages</i>, and their <i>aggregation</i> has to be made via a permutation invariant function like the sum.</p>

## Graph Convolution networks

Our goal is to define a flexible, nonlinear transformation of the node embeddings that is differentiable w.r.t. a set of weight parameters and which maps the embeddings in layer $l$ into corresponding embeddings in layer $l+1$.
<p>As we seen before we can view each layer of processing as having two successive stages: <i>Aggregation</i> and <i>Update</i>.</p>

<p style="text-align:center;"><img src="images/aggupd.png" alt="drawing" width="600" style="text-align:center;"/> </p>

<i>Aggregation</i> must be flexible with respect to:
<ul>
<li>Number of neighbors</li>
<li>Ordering of neighbors</li>
</ul>
it can also contain differentiable parameters as long as it is a differentiable function w.r.t. those parameters.
<p><i>Update</i> operation can also be a differentiable function of a set of learnable parameters</p>
<p>Application of the <i>Aggregate</i> operation followed by the <i>Update</i> operation <b>in parallel</b> for each node in the graph represents <b>one</b> layer of the network.
This framework is called <b>Message-passing graph neural network</b> <i>(Gilmer et al., 2017)</i>.

### Aggregation functions

<b>Sum</b> (no learnable parameters):
$$\sum_{m\in\mathcal{N}(n)}h_m^{(l)}$$
A summation gives stronger influence over nodes that have many neighbors and this can lead to numerical issues, particularly in applications such as social networks where the size of neighborhood can vary by several oreder of magnitude.
<hr/>
<b>Mean</b> (no learnable parameters):
$$\frac{1}{|\mathcal{N}(n)|}\sum_{m\in\mathcal{N}(n)}h_m^{(l)}$$
This normalization solves numerical issues, but discards information about network structure and is provably <b>less</b> powerful than a simple summation <i>(Hamilton, 2020)</i>, and so the choice of whether to use it depends on the relative importance of node features compared to graph structure.

<b>Kipf and Welling, 2016</b> (no learnable parameters):
$$\sum_{m\in\mathcal{N}(n)}\frac{h_m^{(l)}}{\sqrt{|\mathcal{N}(n)||\mathcal{N}{m}|}}$$
takes into account number of neighbors for each neighboring nodes
<hr/>
<b>Element-wise maximum/minimum</b> (no learnable parameters)
<hr/>
<i>Generalizing Aggregation Functions in GNNs: High-Capacity GNNs via Nonlinear Neighborhood Aggregators</i>: <a>https://arxiv.org/pdf/2202.09145</a>



The aggregation functions discussed so far have no learnable parameters. We can introduce such parameters if we first transform each of the embedding vectors from neighboring nodes using an MLP before combining their outputs:
$$\sum_{m\in\mathcal{N}(n)}MLP_\phi(h_m^{(l)})$$
Also, we can transform the aggregation with another MLP:
$$MLP_\theta\left(\sum_{m\in\mathcal{N}(n)}MLP_\phi(h_m^{(l)})\right)$$
in which $MLP_\theta$ and $MLP_\phi$ are shared across layer $l$.
<p>Due to the flexibility of MLPs, this transformation represents a <i>universal approximator</i> for <b>any</b> permutation-invariant function that maps a set of embeddings to a single embedding <i>(Zaheer et. al, 2017)</i></p>
Note that summation can be replaced by other invariant functions.

### Deep sets

A special case of GNNs arises if we consider a graph with <b>no</b> edges, which corresponds simply to an unstructured set of nodes.
<p>In this case if we use the aforementioned aggregation function for each node in the set, in which the summation is taken over all other nodes except the node itself, then we have a general framework for learning functions over unstructured sets of varaibles called <b>deep sets</b></p>.

Since each node in a given layer of the network is updated by aggregating information from its neighbors in the previous layer, this defines a <b>receptive field</b> alogous to the receptive fields of filters used in CNNs.
<p>As information is processed through successive layers, the updates to a given node depende on a steadely increasing fraction of other nodes in earlier layers until the effective receptive field potentially spans the <b>whole graph</b>.</p>
<p style="text-align:center;"><img src="images/field.png" alt="drawing" width="600" style="text-align:center;"/> </p>
<p>However large, sparse graphs may require an excessive number of layers before each output is influenced by every input. Some architectures introduce one or more <b>"super-nodes"</b> that connects directly to all nodes (or a portion of them) in the original graph to ensure fast propagation of information.</p>